In [ ]:
!pip install -r requirements.txt

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
import json
import pandas as pd
import numpy as np
import math as math
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.model_selection as model_selection
import sklearn.tree as tree
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
import time
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
import catboost
from catboost import CatBoostClassifier
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split


In [ ]:
df_train = pd.read_csv('Training_clean.csv')
def root_mean_squared_log_error(y_true, y_pred):
    # Alternatively: sklearn.metrics.mean_squared_log_error(y_true, y_pred) ** 0.5
    assert (y_true >= 0).all() 
    assert (y_pred >= 0).all()
    log_error = np.log1p(y_pred) - np.log1p(y_true)  # Note: log1p(x) = log(1 + x)
    return np.mean(log_error ** 2) ** 0.5

df_test = pd.read_csv('Test_clean.csv')
    


In [ ]:
df_train.head()

,id,seller,price,area_total,area_kitchen,area_living,floor,rooms,layout,ceiling,...,stories,elevator_without,elevator_passenger,elevator_service,parking,garbage_chute,heating,price/area,distance,street
0,0,3,7139520.0,59.2,12.5,31.0,2,2,1,3,...,9,0,1,1,1,1,0,120600.000000,0.239348,Бунинские Луга ЖК
1,1,1,10500000.0,88.0,14.2,48.0,18,3,1,3,...,25,0,1,1,1,1,0,119318.181818,0.129713,улица Радужная
2,2,3,9019650.0,78.5,22.5,40.8,12,3,1,3,...,15,0,1,1,1,1,0,114900.000000,0.121183,Саларьево Парк ЖК
3,3,1,10500000.0,88.0,14.0,48.0,18,3,1,3,...,25,0,1,1,1,1,0,119318.181818,0.129713,улица Радужная
4,4,2,13900000.0,78.0,17.0,35.0,7,2,1,3,...,15,0,1,1,1,0,0,178205.128205,0.217762,улица Сервантеса


In [ ]:
#df_train['floor/stories'] = df_train.floor/df_train.stories
#df_test['floor/stories'] = df_test.floor/df_test.stories

#df_train['distance'] = np.sqrt((37.6 - df_train['longitude'])**2 + (55.75 - df_train['latitude'])**2)
#df_test['distance'] = np.sqrt((37.6 - df_test['longitude'])**2 + (55.75 - df_test['latitude'])**2)

#df_train.drop(['longitude', 'latitude'], axis=1, inplace=True)
#df_test.drop(['longitude', 'latitude'], axis=1, inplace=True)
df_train
df_test

,id,seller,area_total,area_kitchen,area_living,floor,rooms,layout,ceiling,bathrooms_shared,...,material,stories,elevator_without,elevator_passenger,elevator_service,parking,garbage_chute,heating,distance,street
0,23285,3,71.4,13.16,29.44,2,1,1,3,1,...,2,20,1,1,1,1,1,0,0.052411,2-я Черногрязская улица
1,23286,1,54.0,4.00,50.00,4,1,1,3,1,...,2,15,0,1,1,0,0,0,0.216436,улица Сервантеса
2,23287,3,39.4,19.60,11.20,10,1,1,3,1,...,3,9,0,1,1,1,1,0,0.228879,Саларьево Парк ЖК
3,23288,3,60.9,15.68,55.56,5,2,1,3,1,...,2,17,1,1,0,0,1,3,0.217394,улица Александры Монаховой
4,23289,3,34.0,9.00,17.00,14,1,1,3,1,...,2,17,0,1,1,1,1,0,0.345584,улица 3-я Нововатутинская
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9932,33217,3,106.0,19.90,56.70,16,3,1,3,1,...,2,20,0,1,1,0,1,0,0.148935,улица Паршина
9933,33218,1,82.0,8.88,33.22,3,3,1,3,2,...,2,20,0,1,1,1,1,0,0.083756,Западный Порт ЖК
9934,33219,1,49.3,22.06,78.22,15,1,1,0,1,...,2,24,0,1,1,0,1,1,0.225713,Рублевское шоссе
9935,33220,1,38.8,10.50,15.10,14,1,1,3,1,...,1,14,1,1,1,0,1,0,0.062356,улица Архитектора Щусева


In [ ]:
#y1 = df_train['price']

X1_test = df_test.drop(['id'], axis=1)

X1_test_area_total = X1_test.area_total.values # Save area total values from training set so that we can use feature engineering on price/area as measure
df_train['price/area'] = df_train['price']/df_train['area_total']
y1 = df_train['price/area'] # Don't drop area here because we need it for split

X1 = df_train.drop(['price', 'id', 'price/area'], axis=1)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X1, y1, test_size=0.2, random_state=1, shuffle=True)
X_train.shape, X_val.shape, y_train.shape,y_val.shape, X1_test.shape


((18589, 30), (4648, 30), (18589,), (4648,), (9937, 30))

In [ ]:
area_total_train = X_train.area_total.values
area_total_val = X_val.area_total.values
X_train = X_train.drop(['area_total'], axis=1)
X_val = X_val.drop(['area_total'], axis=1)



#seller-0, rooms-3, layout-4, ceiling-5, bathrooms_shared-6, bathrooms_private-7, windows_court-8, windows_street-9, balconies-10, loggias-11, condition-12, phones-13, building_id-14, new-15, district-16, constructed-17, material-18, elevator_without-19, elevator_passenger-20, elevator_service-21, parking-22, garbage_chute-23, heating-24, street-25

In [ ]:
def catboost1(X_trainn, y_trainn, X_testt):
    cat_features = np.where(X_trainn.dtypes != np.float64)[0]
    X_train, X_val, y_train, y_val = train_test_split(X_trainn, y_trainn, test_size=0.2, random_state=1, shuffle=True)
    area_total_train = X_train.area_total.values
    area_total_val = X_val.area_total.values
    X_train = X_train.drop(['area_total'], axis=1)
    X_val = X_val.drop(['area_total'], axis=1)
    X_test = X_testt.drop(['area_total'], axis=1)
    model = CatBoostRegressor(random_state=1, iterations = 2000, cat_features = cat_features, depth=9, learning_rate = 0.1, loss_function='RMSE')
    model.fit(X_train, y_train, eval_set=(X_val, y_val), plot=False)
    #y_pred = model.predict(X_val)*area_total_val
    final_labels = model.predict(X_test.drop(['area_total'], axis=1))
    final_labels = final_labels.astype(float)
    final_labels = final_labels * df_test.area_total.values
    return final_labels


In [ ]:
final_labels = catboost1(X1, y1, X1_test)

#print(f'Train RMSLE: {root_mean_squared_log_error(y_true=y_train, y_pred=y_train_hat) :.4f}')
#print(f'Valid RMSLE: {root_mean_squared_log_error(y_true=y_val, y_pred=y_valid_hat) :.4f}')
print(final_labels.head())

#Train RMSLE: 0.1283
#Valid RMSLE: 0.1761

#Train RMSLE: 0.0489
#Valid RMSLE: 0.1451

#Train RMSLE: 0.0071
#Valid RMSLE: 0.0187

CatBoostError: Invalid cat_features[25] = 29 value: index must be < 29.

In [ ]:
submission = pd.DataFrame()
submission['id'] = df_test.id
submission['price_prediction'] = final_labels # Predict on non-nan entries
 # Fill missing entries with mean predictor
print(f'Generated {len(submission)} predictions')

# Export submission to csv with headers
#submission.to_csv('sample_submission.csv', index=False)

submission.to_csv('sample_submission9.csv', index=False)

Generated 9937 predictions


In [ ]:
!pip install shap

     |████████████████████████████████| 371 kB 5.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.3 MB 45.5 MB/s 
     |████████████████████████████████| 26.3 MB 47.6 MB/s 
     |████████████████████████████████| 15.4 MB 49.1 MB/s 
  Created wheel for shap: filename=shap-0.40.0-cp39-cp39-linux_x86_64.whl size=435079 sha256=98a5f419fb35aafbeaf29dad8b5cce093ef41e47b865c45a5fbc295ef69b5676
  Stored in directory: /root/.cache/pip/wheels/bc/27/26/8c2ff91c64e876300f0a95bd2dd560cedf45bdb92b5fcd4400
Successfully built shap
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.4
    Not uninstalling numpy at /shared-libs/python3.9/py/lib/python3.9/site-packages, outside environment /root/venv
    Can't uninstall 'numpy'. No files were found to uninstall.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' comman

In [ ]:
shap.summary_plot(shap_values, X_train)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=99c1e030-46bc-40fa-835c-0661eae0d488' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>